### This notebook demonstrates how to evaluate accuracy

In [1]:
from sklearn.neighbors import NearestNeighbors
from TfidfVectorizer import TfidfVectorizer
from tqdm import tqdm
from utils import *

# don't forget to set clean=True to clean the data!
data = load_data_from_json('data/data_5d.json', clean=True, use_examples=True)
data_test = load_data_from_json('data/test_200.json')

100%|██████████| 103874/103874 [00:03<00:00, 31158.40it/s]


In [2]:
data_train, data_seen_500, data_unseen_500 = split_seen_unseen(data)
print(len(list(itertools.chain(*data_train.values()))))
print(len(list(itertools.chain(*data_seen_500.values()))))
print(len(list(itertools.chain(*data_unseen_500.values()))))

576739
500
500


### Fit training data

In [3]:
# fit tfidf vectorizer
vectorizer = TfidfVectorizer(stop_words=None, stemmer=None, norm=False)
X_tfidf = vectorizer.fit_transform(data_train) # <--- fit train data here
words = vectorizer.words

# fit knn
knn = NearestNeighbors(metric='cosine')
knn.fit(X_tfidf)

100%|██████████| 96213/96213 [00:23<00:00, 4026.42it/s] 


NearestNeighbors(metric='cosine')

### Test seen data

In [4]:
y_pred = []
y_gold = []
for word, defi in tqdm(data_seen_500.items()): # <--- use seen_500 data here
    query = list(defi)[0]
    query = vectorizer.transform(query)

    prediction = search(query, knn, words, n=1000)

    y_pred.append(prediction)
    y_gold.append(word)

evaluate(y_pred, y_gold)

100%|██████████| 36/36 [00:03<00:00,  9.46it/s]

acc@1: 0.61
acc@10: 0.89
acc@100: 0.97
median rank: 0
standard error of mean rank: 6


(0.6111111111111112,
 0.8888888888888888,
 0.9722222222222222,
 0.0,
 5.896847056912705)

### Test unseen data

In [5]:
y_pred = []
y_gold = []
for word, defi in tqdm(data_unseen_500.items()): # <--- use unseen_500 data here
    query = list(defi)[0]
    query = vectorizer.transform(query)

    prediction = search(query, knn, words, n=1000)

    y_pred.append(prediction)
    y_gold.append(word)

evaluate(y_pred, y_gold)

100%|██████████| 481/481 [00:47<00:00, 10.10it/s]


acc@1: 0.09
acc@10: 0.25
acc@100: 0.55
median rank: 70
standard error of mean rank: 17


(0.09355509355509356,
 0.25363825363825365,
 0.553014553014553,
 70.0,
 16.901458438573744)

### Test description set (200) data

In [6]:
# fit tfidf vectorizer
vectorizer = TfidfVectorizer(stop_words=None, stemmer=None, norm=False)
X_tfidf = vectorizer.fit_transform(data) # <--- fit all data here
words = vectorizer.words

# fit knn
knn = NearestNeighbors(metric='cosine')
knn.fit(X_tfidf)

y_pred = []
y_gold = []
for word, defi in tqdm(data_test.items()): # <--- use desc_200 data here
    query = list(defi)[0]
    query = vectorizer.transform(query)

    prediction = search(query, knn, words, n=1000)

    y_pred.append(prediction)
    y_gold.append(word)

evaluate(y_pred, y_gold)

100%|██████████| 200/200 [00:20<00:00,  9.92it/s]

acc@1: 0.02
acc@10: 0.06
acc@100: 0.28
median rank: 316
standard error of mean rank: 29


(0.02, 0.06, 0.285, 316.5, 28.616604585450037)